In [1]:
import cv2
import mediapipe as mp
import pyautogui
import sys
sys.path.append(r"C:\Users\ASUS\virtual mouse")  # Add path to sys.path

import util

In [2]:
screen_width, screen_height = pyautogui.size()

mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    max_num_hands = 1,
    
)

In [3]:
def find_finger_tip(processed):
    if processed.multi_hand_landmarks:  
        hand_landmarks = processed.multi_hand_landmarks[0]  # ✅ Use '=' instead of 'in'
        return hand_landmarks.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP]  # ✅ Use '[]' instead of '{}'

    return None  # ✅ Proper indentation


In [4]:
def  move_mouse(index_finger_tip):
    if index_finger_tip is not None:
            x = int(index_finger_tip.x * screen_width)
            x = int(index_finger_tip.x * screen_height)
            pyautogui.moveTo(x,y)

In [5]:
def detect_gestures(frame, landmark_list, processed):
    if len(landmark_list) > 21:  # ✅ Use 'landmark_list' (not 'landmarks_list')

        index_finger_tip = find_finger_tip(processed)
        print(index_finger_tip)

        # ✅ Corrected distance calculation (between thumb and index finger tip)
        thumb_index_dist = util.get_distance(landmark_list[4], landmark_list[8])  

        # ✅ Corrected angle calculation
        if thumb_index_dist < 50 and util.get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) > 90:
            move_mouse(index_finger_tip)  # ✅ Ensure index_finger_tip contains (x, y) coordinates


In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import numpy as np
import time

# Disable PyAutoGUI fail-safe
pyautogui.FAILSAFE = False  

# Initialize Mediapipe Hands
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Get screen size
screen_width, screen_height = pyautogui.size()

click_threshold = 20  # Distance between index and thumb for left click
right_click_threshold = 20  # Distance between middle and thumb for right click
was_pinch = False  # Track if pinch was previously detected
was_right_pinch = False  # Track if right-click pinch was detected
last_click_time = 0  # Track last click time to prevent double clicks
click_delay = 0.2  # Delay to prevent unintended double clicks

active_hand_id = None  # Track the active hand
cursor_locked = False  # Prevent cursor movement during pinch
SENSITIVITY = 2.0  # Adjusted cursor movement sensitivity for smoother control

prev_hand_x, prev_hand_y = None, None  # Track previous hand position

# Function to check if ring and pinky fingers are closed
def are_ring_pinky_closed(landmarks):
    return landmarks[16][1] > landmarks[13][1] and landmarks[20][1] > landmarks[17][1]

# Gesture detection function
def detect_gestures(frame, landmarks, hand_id):
    global was_pinch, was_right_pinch, last_click_time, active_hand_id, cursor_locked
    global prev_hand_x, prev_hand_y

    # Ignore new hands until the active one disappears
    if active_hand_id is None:
        active_hand_id = hand_id
    elif active_hand_id != hand_id:
        return

    # Check if ring and pinky fingers are closed
    fingers_closed = are_ring_pinky_closed(landmarks)
    if not fingers_closed:
        prev_hand_x, prev_hand_y = None, None  # Reset hand tracking when fingers are open
        return  # Stop movement if fingers are open

    # Get fingertip positions
    index_tip = landmarks[8]
    thumb_tip = landmarks[4]
    middle_tip = landmarks[12]

    # Click detection (Pinch Gesture for Left Click)
    pinch_detected = np.linalg.norm(np.array(index_tip) - np.array(thumb_tip)) < click_threshold
    right_pinch_detected = np.linalg.norm(np.array(middle_tip) - np.array(thumb_tip)) < right_click_threshold

    current_time = time.time()
    if pinch_detected and not was_pinch and (current_time - last_click_time > click_delay):
        pyautogui.click()  # Left Click
        last_click_time = current_time
        cursor_locked = True  # Lock cursor movement during click
    elif right_pinch_detected and not was_right_pinch and (current_time - last_click_time > click_delay):
        pyautogui.rightClick()  # Right Click
        last_click_time = current_time
        cursor_locked = True  # Lock cursor movement during click
    elif not pinch_detected and not right_pinch_detected:
        cursor_locked = False  # Unlock cursor when pinch is released
    
    was_pinch = pinch_detected  # Update pinch state
    was_right_pinch = right_pinch_detected

    # Only move cursor if not locked
    if not cursor_locked:
        if prev_hand_x is None or prev_hand_y is None:
            prev_hand_x, prev_hand_y = index_tip[0], index_tip[1]  # Initialize hand tracking
        
        dx = (index_tip[0] - prev_hand_x) / frame.shape[1] * screen_width
        dy = (index_tip[1] - prev_hand_y) / frame.shape[0] * screen_height
        
        cursor_x = max(1, min(screen_width - 1, int(pyautogui.position()[0] + dx * SENSITIVITY)))
        cursor_y = max(1, min(screen_height - 1, int(pyautogui.position()[1] + dy * SENSITIVITY)))
        
        pyautogui.moveTo(cursor_x, cursor_y, duration=0.02)
        prev_hand_x, prev_hand_y = index_tip[0], index_tip[1]

# Main function
def main():
    cap = cv2.VideoCapture(0)
    global active_hand_id
    
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            continue
        
        frame = cv2.flip(frame, 1)  # Mirror image
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        if results.multi_hand_landmarks:
            if active_hand_id is None:
                active_hand_id = 0
            
            hand_landmarks = results.multi_hand_landmarks[active_hand_id]
            landmarks_list = [(int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])) for lm in hand_landmarks.landmark]
            detect_gestures(frame, landmarks_list, active_hand_id)
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        else:
            active_hand_id = None

        cv2.imshow("Virtual Mouse", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):  
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
